In [2]:
######################Generating CSV file

import cv2
import csv
import glob
import pandas as pd
from sklearn.utils import shuffle
import os

# Specify the CSV file name
csv_file = 'bottom_part.csv'

# Check if the file exists and delete it
if os.path.exists(csv_file):
    os.remove(csv_file)
    print(f"Deleted existing {csv_file} file.")

# Initialize the header
header = ["label"]
for i in range(0, 784):
    header.append("pixel" + str(i))

# Create the CSV file and write the header
with open(csv_file, 'a') as f:
    writer = csv.writer(f)
    writer.writerow(header)

# List of character folders
character_folders = ["o","u"]

# Loop through character folders
for character_folder in character_folders:
    dirList = glob.glob("baybayin_images/" + character_folder + "/*.jpg")

    # Get label from folder name
    label = character_folder

    for img_path in dirList:
        im = cv2.imread(img_path)
        im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        im_gray = cv2.GaussianBlur(im_gray, (1, 1), 0)
        # im_gray = cv2.GaussianBlur(im_gray, (15, 15), 0)
        roi = cv2.resize(im_gray, (28, 28), interpolation=cv2.INTER_AREA)

        data = []
        data.append(label)

        rows, cols = roi.shape

        # Threshold using Otsu's method
        _, thresholded = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        for i in range(rows):
            for j in range(cols):
                k = thresholded[i, j]
                data.append(k)

        # Print data for debugging
        print(data)

        with open('bottom_part.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(data)

# Load and shuffle the dataset
dataset = pd.read_csv('bottom_part.csv')
shuffled_dataset = shuffle(dataset)
print(shuffled_dataset)


KeyboardInterrupt: 

In [ ]:

###########################################################Separating labels and pixels into y and x axis respectively


import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Load and shuffle the dataset
dataset = pd.read_csv('bottom_part.csv')
shuffled_dataset = shuffle(dataset)

# Separate dependent and independent variables
X = shuffled_dataset.drop(["label"], axis=1)
Y = shuffled_dataset["label"]


In [ ]:

################################################################Show an image on how it look like in grayscale with 2 values, black and whit

%matplotlib inline
idx = 150
img = X.loc[idx].values.reshape(28, 28)
print(Y[idx])
plt.imshow(img, cmap='gray')  # Use 'gray' colormap for grayscale images
plt.show()


In [ ]:

####################################################################Splitting data set into training and testing data

# Train-Test split
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2)

In [ ]:
####################################################################Training the model 

import joblib
from sklearn.svm import SVC
import os

# Specify the model file name
model_file = 'model/baybayinbottom_model'

# Check if the file exists and delete it
if os.path.exists(model_file):
    os.remove(model_file)
    print(f"Deleted existing {model_file} file.")


# Fit the model using SVC
classifier = SVC(kernel="linear", random_state=6)
classifier.fit(train_x, train_y)

# Save the model using joblib
joblib.dump(classifier, "model/baybayinbottom_model")
                   

In [ ]:

####################################################################Calculate accuracy

from sklearn import metrics

# Calculate accuracy
prediction = classifier.predict(test_x)
accuracy = metrics.accuracy_score(prediction, test_y)
print("Accuracy =", accuracy)

                   

In [3]:
import joblib
import cv2
import numpy as np
import glob
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Load the trained model
model = joblib.load("model/baybayinbottom_model")

# Folder containing test images
test_images_folder = "test_images/"

# Loop through test images
for img_path in glob.glob(test_images_folder + "*.jpg"):
    # Read the image
    im = cv2.imread(img_path)
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im_gray = cv2.GaussianBlur(im_gray, (1, 1), 0)

    # Threshold the image
    _, im_th = cv2.threshold(im_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    roi = cv2.resize(im_th, (28, 28), interpolation=cv2.INTER_AREA)

    rows, cols = roi.shape

    X = []

    # Fill the data array with pixels one by one.
    for i in range(rows):
        for j in range(cols):
            k = roi[i, j]
            X.append(k)

    # Make a prediction
    predictions = model.predict([X])
    predicted_label = predictions[0]

    # Display the result on the console
    print(f"Image: {img_path}, Prediction: {predicted_label}")


Image: test_images\bottomCanvas_0_output.jpg, Prediction: o
Image: test_images\bottomCanvas_7_output.jpg, Prediction: u
Image: test_images\c (1).jpg, Prediction: o
Image: test_images\c (2).jpg, Prediction: o
Image: test_images\c (3).jpg, Prediction: o
Image: test_images\c (4).jpg, Prediction: o
Image: test_images\c (5).jpg, Prediction: o
Image: test_images\v (1).jpg, Prediction: u
Image: test_images\v (2).jpg, Prediction: u
Image: test_images\v (3).jpg, Prediction: u
Image: test_images\v (4).jpg, Prediction: u
Image: test_images\v (5).jpg, Prediction: u
Image: test_images\v (6).jpg, Prediction: u
